# Generating Bug Free Leetcode Solutions

!!! note
    To download this tutorial as a Jupyter notebook, click [here](https://github.com/guardrails-ai/guardrails/blob/main/docs/examples/bug_free_python_code.ipynb).

In this example, we want to solve String Maniuplation leetcode problems such that the code is bug free.

We make the assumption that:

1. We don't need any external libraries that are not already installed in the environment.
2. We are able to execute the code in the environment.

## Objective

We want to generate bug-free code for solving leetcode problems. In this example, we don't account for semantic bugs, only for syntactic bugs.

In short, we want to make sure that the code can be executed without any errors.

## Step 1: Generating `RAIL` Spec

First, we install the validators and packages we need to make sure generated python is valid. Then, we generate a RAIL spec as a pydantic model.

In [1]:
!guardrails hub install hub://reflex/valid_python --quiet

Installing hub://reflex/valid_python...
✅Successfully installed reflex/valid_python!




In [2]:
from pydantic import BaseModel, Field
from guardrails.hub import ValidPython

prompt = """
Given the following high level leetcode problem description, write a short Python code snippet that solves the problem.

Problem Description:
${leetcode_problem}

${gr.complete_xml_suffix}"""

class BugFreePythonCode(BaseModel):
    python_code: str = Field(validators=[ValidPython(on_fail="reask")])

    class Config:
        arbitrary_types_allowed = True

## Step 2: Create a `Guard` object with the RAIL Spec

We create a `gd.Guard` object that will check, validate and correct the generated code. This object:

1. Enforces the quality criteria specified in the RAIL spec (i.e. bug free code).
2. Takes corrective action when the quality criteria are not met (i.e. reasking the LLM).
3. Compiles the schema and type info from the RAIL spec and adds it to the prompt.

In [3]:
import guardrails as gd

from rich import print

Or from the pydantic model:

In [4]:
guard = gd.Guard.from_pydantic(output_class=BugFreePythonCode)

## Step 3: Wrap the LLM API call with `Guard`

In [19]:
import os
os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"

leetcode_problem = """
Given a string s, find the longest palindromic substring in s. You may assume that the maximum length of s is 1000.
"""

response = guard(
    model="gpt-4o",
    prompt=prompt,
    prompt_params={"leetcode_problem": leetcode_problem},
    max_tokens=2048,
    temperature=0,
)
print(response)

ValidationOutcome(
    call_id='4892992976',
    raw_llm_output='```json\n{\n  "python_code": "def longest_palindrome(s: str) -> str:\\n    n = len(s)\\n    if 
n == 0:\\n        return \'\'\\n    start, max_length = 0, 1\\n    for i in range(n):\\n        if i - max_length 
>= 1 and s[i-max_length-1:i+1] == s[i-max_length-1:i+1][::-1]:\\n            start = i - max_length - 1\\n         
max_length += 2\\n            continue\\n        if i - max_length >= 0 and s[i-max_length:i+1] == 
s[i-max_length:i+1][::-1]:\\n            start = i - max_length\\n            max_length += 1\\n    return 
s[start:start + max_length]"\n}\n```',
    validated_output={
        'python_code': "def longest_palindrome(s: str) -> str:\n    n = len(s)\n    if n == 0:\n        return ''\n
start, max_length = 0, 1\n    for i in range(n):\n        if i - max_length >= 1 and s[i-max_length-1:i+1] == 
s[i-max_length-1:i+1][::-1]:\n            start = i - max_length - 1\n            max_length += 2\n            
continue\n        if i - max_length >= 0 and s[i-max_length:i+1] == s[i-max_length:i+1][::-1]:\n            start =
i - max_length\n            max_length += 1\n    return s[start:start + max_length]"
    },
    reask=None,
    validation_passed=True,
    error=None
)

The `Guard` object compiles the output schema and adds it to the prompt. We can see the final prompt below:

In [20]:
print(guard.history.last.compiled_prompt)

Given the following high level leetcode problem description, write a short Python code snippet that solves the 
problem.

Problem Description:

Given a string s, find the longest palindromic substring in s. You may assume that the maximum length of s is 1000.



Given below is XML that describes the information to extract from this document and the tags to extract it into.

<output>
  <string format="reflex/valid_python" name="python_code" required="true"></string>
</output>

ONLY return a valid JSON object (no other text is necessary), where the key of the field in JSON is the `name` 
attribute of the corresponding XML, and the value is of the type specified by the corresponding XML's tag. The JSON
MUST conform to the XML format, including any types and format requests e.g. requests for lists, objects and 
specific types. Be correct and concise. If you are unsure anywhere, enter `null`.

Here are examples of simple (XML, JSON) pairs that show the expected behavior:
- `<string name='foo' format='two-words lower-case' />` => `{'foo': 'example one'}`
- `<list name='bar'><string format='upper-case' /></list>` => `{"bar": ['STRING ONE', 'STRING TWO', etc.]}`
- `<object name='baz'><string name="foo" format="capitalize two-words" /><integer name="index" format="1-indexed" 
/></object>` => `{'baz': {'foo': 'Some String', 'index': 1}}`

Running the cell above returns:
1. The raw LLM text output as a single string.
2. A dictionary where the key is `python_code` and the value is the generated code.

In [21]:
print(response.validated_output)

{
    'python_code': "def longest_palindrome(s: str) -> str:\n    n = len(s)\n    if n == 0:\n        return ''\n    
start, max_length = 0, 1\n    for i in range(n):\n        if i - max_length >= 1 and s[i-max_length-1:i+1] == 
s[i-max_length-1:i+1][::-1]:\n            start = i - max_length - 1\n            max_length += 2\n            
continue\n        if i - max_length >= 0 and s[i-max_length:i+1] == s[i-max_length:i+1][::-1]:\n            start =
i - max_length\n            max_length += 1\n    return s[start:start + max_length]"
}

Here's the generated code:

In [22]:
if response.validated_output is not None:
    print(response.validated_output["python_code"])
elif response.error is not None:
    print(response.error)

def longest_palindrome(s: str) -> str:
    n = len(s)
    if n == 0:
        return ''
    start, max_length = 0, 1
    for i in range(n):
        if i - max_length >= 1 and s == s[::-1]:
            start = i - max_length - 1
            max_length += 2
            continue
        if i - max_length >= 0 and s == s[::-1]:
            start = i - max_length
            max_length += 1
    return s

We can confirm that the code is bug free by executing the code in the environment.

In [23]:
try:
    exec(response.validated_output["python_code"])
    print("Success!")
except Exception as e:
    print("Failed!")

Success!